Work Flow:
Data - Data pre processing(Clean the data etc) - Feature Extraction(Textual data-numerical data) - Similarity confidence score - Cosine Similarity - Get input from user - Suggest rest of movies 
(No model is used only cosine similarity is used)



Importing dependencies


In [2]:
import numpy as np
import pandas as pd
import difflib

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection 


In [7]:
movies_data = pd.read_csv('/content/movies.csv')

In [8]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [9]:
movies_data.shape

(4803, 24)

In [10]:
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [11]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [12]:
combined_features = movies_data['genres']+movies_data['keywords']+movies_data['tagline']+movies_data['cast']+movies_data['director']

In [13]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [14]:
vectorizer = TfidfVectorizer()

In [15]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [16]:
print(feature_vectors)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

In [18]:
similarity = cosine_similarity(feature_vectors)

In [19]:
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [20]:
print(similarity.shape)

(4803, 4803)


In [21]:
movie_name = input('Enter your favourite movie: ')

Enter your favourite movie: Forest Gump


In [22]:
list_of_all_titles = movies_data['title'].tolist()

In [23]:
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [24]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

In [25]:
print(find_close_match)

['Forrest Gump', 'Firestorm']


In [26]:
close_match = find_close_match[0]
print(close_match)

Forrest Gump


In [28]:
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

809


In [29]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [30]:
print(similarity_score)

[(0, 0.029035321308325257), (1, 0.047397994976484545), (2, 0.023623222581181734), (3, 0.04492693669707403), (4, 0.021564991962301154), (5, 0.006477301015248168), (6, 0.0), (7, 0.023053631263943993), (8, 0.02239967304184119), (9, 0.023127462591267127), (10, 0.026261995277997007), (11, 0.0), (12, 0.0), (13, 0.009143414426896533), (14, 0.05989271275459622), (15, 0.00843836935822501), (16, 0.021948167789170747), (17, 0.0), (18, 0.026432143971164446), (19, 0.02208591339557235), (20, 0.00609979207560244), (21, 0.0), (22, 0.015065225730707753), (23, 0.027707435468390884), (24, 0.030804868159040502), (25, 0.012732445454292333), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.009229861943397468), (30, 0.012618183877106445), (31, 0.027427779279772878), (32, 0.008041277443435087), (33, 0.023265197237162943), (34, 0.01726711263176703), (35, 0.0), (36, 0.0116271643422748), (37, 0.012556892657930939), (38, 0.023451723713451063), (39, 0.0), (40, 0.014740166379058589), (41, 0.026493372586398254), (42, 0.04334

In [31]:
len(similarity_score)

4803

Sorting


In [32]:
sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(809, 1.0), (690, 0.16347771256387467), (1812, 0.13892891443665406), (3434, 0.12251122313597484), (384, 0.11518079889001243), (635, 0.11493438292562252), (2757, 0.11144468215220743), (2946, 0.11032261648556362), (628, 0.10909207105640725), (998, 0.10893089394341082), (2980, 0.10751899507624837), (638, 0.10555884307521916), (1259, 0.105522608245951), (1957, 0.10499665232681625), (1239, 0.10446536143595152), (548, 0.10304336532244734), (723, 0.10301845933542719), (2906, 0.10282683905161619), (871, 0.10044348500605037), (3019, 0.0985345594494643), (643, 0.09821507502542806), (1562, 0.09788464869384432), (2071, 0.09499538584179859), (4573, 0.09357677369805187), (238, 0.09275946577149535), (2314, 0.09260418613846429), (4010, 0.09071049657513802), (4330, 0.09047986169954075), (1272, 0.08962717863499756), (928, 0.08707222637852075), (4062, 0.08649869831894749), (365, 0.08529969776749777), (3286, 0.08427604159600435), (151, 0.08393371770841437), (1159, 0.08338677805863827), (373, 0.0831380121

In [35]:
print('Movies suggested for you: \n')
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i, '.', title_from_index)
    i+=1

Movies suggested for you: 

1 . Forrest Gump
2 . The Green Mile
3 . Rain Man
4 . A Home at the End of the World
5 . Cast Away
6 . Apollo 13
7 . Letters from Iwo Jima
8 . What's Eating Gilbert Grape
9 . Saving Private Ryan
10 . The Big Bounce
11 . The Purge: Election Year
12 . You've Got Mail
13 . Memoirs of an Invisible Man
14 . Dangerous Liaisons
15 . The Musketeer
16 . Snake Eyes
17 . The Happening
18 . Dancin' It's On
19 . Gigli
20 . A Prairie Home Companion
21 . Space Cowboys
22 . Rugrats in Paris: The Movie
23 . Silver Linings Playbook
24 . As It Is in Heaven
25 . Teenage Mutant Ninja Turtles
26 . Big
27 . That Thing You Do!
28 . Lake Mungo
29 . Impostor


RECOMMENDATION SYSTEM


In [36]:
movie_name = input('Enter your favourite movie: ')
list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse = True)

print('Movies suggested for you: \n')
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i, '.', title_from_index)
    i+=1

Enter your favourite movie: La La Land
Movies suggested for you: 

1 . Stake Land
2 . My Summer of Love
3 . In the Cut
4 . The Wraith
5 . Training Day
6 . Limbo
7 . The Wolverine
8 . Halloween 4: The Return of Michael Myers
9 . The Innkeepers
10 . Halloween
11 . Ripley's Game
12 . Logan's Run
13 . Top Gun
14 . Daybreakers
15 . The Rage: Carrie 2
16 . The Helix... Loaded
17 . Halloween 5: The Revenge of Michael Myers
18 . Backmask
19 . At First Sight
20 . Dogtooth
21 . American Outlaws
22 . Hardball
23 . The American President
24 . Witness
25 . Crank: High Voltage
26 . Licence to Kill
27 . Warrior
28 . The Boys from Brazil
29 . Antitrust
